In [1]:
pip install numpy pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 1.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pymongo]m2/3 [pymongo]n]
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Imports
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, TimestampType
import signal
import time
import sys

In [ ]:
# Initialize Spark Session with proper configurations
spark = SparkSession.builder \
    .appName("DockerSparkToLocalMongo") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1,"
            "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .config("spark.mongodb.output.uri", "mongodb://host.docker.internal:27017/movie_lens.recommendations") \
    .config("spark.sql.streaming.checkpointLocation", "/tmp/checkpoint") \
    .getOrCreate()


# Define schema for incoming Kafka messages
rating_schema = StructType([
    StructField("userId", IntegerType()),  # Changed from user_id
    StructField("movieId", IntegerType()),  # Changed from movie_id
    StructField("rating", FloatType()),
    StructField("timestamp", TimestampType())
])

# Function to process each batch of data
def process_batch(batch_df, batch_id):
    if not batch_df.isEmpty():
        try:
            # Add processing metadata
            result_df = batch_df.withColumn("processing_time", F.current_timestamp()) \
                              .withColumn("batch_id", F.lit(batch_id))
            
            # Write to MongoDB
            (result_df.write
                .format("mongo")
                .mode("append")
                .option("database", "movie_lens")
                .option("collection", "recommendations")
                .save())
            
            print(f"Successfully processed batch {batch_id} with {batch_df.count()} records")
        except Exception as e:
            print(f"Error processing batch {batch_id}: {str(e)}", file=sys.stderr)

# Create Kafka source stream
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "namenode:9092") \
    .option("subscribe", "movie_rating") \
    .option("startingOffsets", "latest") \
    .option("failOnDataLoss", "false") \
    .option("maxOffsetsPerTrigger", "1000") \
    .load()

# Parse the JSON data from Kafka
processed_df = df.selectExpr("CAST(value AS STRING)") \
                .select(F.from_json(F.col("value"), rating_schema).alias("data")) \
                .select("data.*")

# Start the streaming query
query = processed_df.writeStream \
    .foreachBatch(process_batch) \
    .outputMode("update") \
    .option("checkpointLocation", "file:///tmp/checkpoint_movies") \
    .trigger(processingTime='10 seconds') \
    .start()

# Graceful shutdown handler
def handle_shutdown(signum, frame):
    print("\nShutting down gracefully...")
    query.stop()
    spark.stop()
    sys.exit(0)

signal.signal(signal.SIGINT, handle_shutdown)
signal.signal(signal.SIGTERM, handle_shutdown)

# Monitoring loop
try:
    while query.isActive:
        progress = query.lastProgress
        if progress:
            print(f"Batch ID: {progress['batchId']}, "
                  f"Input rows: {progress['numInputRows']}, "
                  f"Processed: {progress['processedRowsPerSecond']:.1f} rows/sec")
        time.sleep(5)
except Exception as e:
    print(f"Streaming query failed: {str(e)}", file=sys.stderr)
    handle_shutdown(None, None)

25/05/02 10:15:05 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/02 10:15:08 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/02 10:15:16 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 10019 milliseconds
[Stage 0:>                                                          (0 + 1) / 1]

Batch ID: 0, Input rows: 0, Processed: 0.0 rows/sec


[Stage 1:>                                                          (0 + 1) / 1]

Batch ID: 0, Input rows: 0, Processed: 0.0 rows/sec
Batch ID: 0, Input rows: 0, Processed: 0.0 rows/sec
Batch ID: 0, Input rows: 0, Processed: 0.0 rows/sec
Batch ID: 0, Input rows: 0, Processed: 0.0 rows/sec
Batch ID: 0, Input rows: 0, Processed: 0.0 rows/sec
Batch ID: 0, Input rows: 0, Processed: 0.0 rows/sec


25/05/02 10:15:56 ERROR Executor: Exception in task 0.0 in stage 1.0 (TID 1)
com.mongodb.MongoTimeoutException: Timed out after 30000 ms while waiting to connect. Client view of cluster state is {type=UNKNOWN, servers=[{address=host.docker.internal:27017, type=UNKNOWN, state=CONNECTING, exception={com.mongodb.MongoSocketOpenException: Exception opening socket}, caused by {java.net.ConnectException: Connection refused (Connection refused)}}]
	at com.mongodb.internal.connection.BaseCluster.getDescription(BaseCluster.java:177)
	at com.mongodb.internal.connection.SingleServerCluster.getDescription(SingleServerCluster.java:41)
	at com.mongodb.client.internal.MongoClientDelegate.getConnectedClusterDescription(MongoClientDelegate.java:147)
	at com.mongodb.client.internal.MongoClientDelegate.createClientSession(MongoClientDelegate.java:98)
	at com.mongodb.client.internal.MongoClientDelegate$DelegateOperationExecutor.getClientSession(MongoClientDelegate.java:278)
	at com.mongodb.client.internal

...

Batch ID: 12, Input rows: 11, Processed: 2.9 rows/sec


Batch ID: 13, Input rows: 11, Processed: 3.7 rows/sec


Batch ID: 13, Input rows: 11, Processed: 3.7 rows/sec


Batch ID: 14, Input rows: 9, Processed: 2.9 rows/sec


Batch ID: 14, Input rows: 9, Processed: 2.9 rows/sec
Batch ID: 15, Input rows: 11, Processed: 3.5 rows/sec
